# 기본데이터 만들기 (처음 1번만)

In [165]:
# ======================================================================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy as cp
# ======================================================================================================================

scaler = MinMaxScaler()
random.seed(54654)

matplotlib.rcParams['font.family']='Malgun Gothic'   # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False

label = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_label.csv')



# ======================================================================================================================
print("process - 1")

activity = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_activity.csv')

# activity acc_id로 groupby
# - 평균을 내지 않은 이유 : 평균을 냈을 경우 캐릭터는 많지만
#   한 캐릭터만으로 활동한 사람의 정보가 과소평가 될 가능성이 있음
activity = activity.groupby(['acc_id'], as_index = False).sum()
activity.drop(columns = ['day','char_id'], inplace = True)
# print(activity.head())



# ======================================================================================================================


combat = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_combat.csv')

combat.drop(columns = ['day', 'server', 'char_id', 'class'], inplace = True)
combat_a = combat.groupby(['acc_id'], as_index = False).sum()
combat_a.drop(columns = 'level', inplace = True)

combat_b = combat.groupby(['acc_id'], as_index = False).max()
#  combat.groupby('acc_id', as_index = False).sum().sort_values('acc_id')

# acc_id 기준으로 정리

combat_b = combat_b[['acc_id', 'level']]
combat = pd.merge(combat_b, combat_a, how = 'inner', on = 'acc_id')



# ======================================================================================================================


payment = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_payment.csv')

payment = payment.groupby('acc_id', as_index = False).sum()
payment.drop(columns = 'day', inplace = True)

payment.rename(columns = {'amount_spent' : 'amount_spent_pay'}, inplace = True)


# ======================================================================================================================


trade = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_trade.csv')

# 거래에 참여한 횟수를 기준으로

# 판매자로서 활동한 acc_id
trade_a = trade.groupby('source_acc_id', as_index = False).count()
trade_a = trade_a[['source_acc_id', 'day']]

# 구매자로서 활동한 acc_id
trade_b = trade.groupby('target_acc_id', as_index = False).count()
trade_b = trade_b[['target_acc_id', 'day']]

x = trade_a['day'].sum() - trade_b['day'].sum()
# print(x) # 0

trade_a.rename(columns={'source_acc_id':'acc_id',
                        'day':'sell_item_cnt'}, inplace=True)
trade_b.rename(columns={'target_acc_id':'acc_id',
                        'day':'buy_item_cnt'}, inplace=True)

trade = pd.merge(trade_a, trade_b, how = 'outer', on = 'acc_id')

# 실제 데이터 검색
# trade[trade['source_acc_id'] == 6].count()

# 데이터 검증
# trade_a[trade_a['source_acc_id'] == 6]


# ======================================================================================================================


pledge = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_pledge.csv')

ple_1 = pledge.groupby(['server', 'pledge_id', 'day'], as_index = False).mean()
ple_1.drop(columns = ['acc_id', 'char_id', 'day'], inplace = True)
ple_1 = ple_1.groupby(['server', 'pledge_id'], as_index = False).sum()

ple_a = pledge.groupby(['acc_id', 'char_id', 'server', 'pledge_id', 'day'], as_index = False).mean()
ple_a = ple_a.groupby(['acc_id', 'char_id', 'server', 'pledge_id'], as_index = False).sum()
ple_a = ple_a[['acc_id', 'char_id', 'server', 'pledge_id']]

pledge = pd.merge(ple_a, ple_1, how = 'outer', on = ['server', 'pledge_id'])
pledge.drop(columns = ['char_id', 'server', 'pledge_id'], inplace = True)
pledge_total = pledge.groupby(['acc_id'], as_index = False).mean()

pledge_total.rename(columns = {'random_attacker_cnt' : 'random_attacker_cnt_plg',
                              'random_defender_cnt' : 'random_defender_cnt_plg',
                              'same_pledge_cnt' : 'same_pledge_cnt_plg',
                              'temp_cnt' : 'temp_cnt_plg',
                              'etc_cnt':'etc_cnt_plg'}, inplace = True)

# ple_1[(ple_1['pledge_id'] == 25467) & (ple_1['server'] == 'aq')]
# ple_a[(ple_a['pledge_id'] == 25467) & (ple_a['server'] == 'aq')]


# ======================================================================================================================
print("process - 2")

# label + activity
label_a = pd.merge(label, activity, how = 'outer', on = 'acc_id')

# (label + activity) + combat
label_b = pd.merge(label_a, combat, how = 'outer', on = 'acc_id')

# (label + activity + combat) + payment
label_c = pd.merge(label_b, payment, how = 'outer', on = 'acc_id')

# (label + activity + combat + payment) + trade
label_d = pd.merge(label_c, trade, how = 'outer', on = 'acc_id')
label_d = label_d[label_d['survival_time'] >= 1]

# (label + activity + combat + payment + trade) + pledge_total
label_z = pd.merge(label_d, pledge_total, how = 'outer', on = 'acc_id')

data = label_z.fillna(0)
print(data.columns)

data[['playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']] = \
    scaler.fit_transform(data[['playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']])

data = data.fillna(0)
print(data.columns)


data.to_csv('AllMergeFromDonny.csv')
# ======================================================================================================================


process - 1
process - 2
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', '

# 데이터 불러오기

In [18]:
# ======================================================================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy as cp
# ======================================================================================================================
scaler = MinMaxScaler()
random.seed(54654)

matplotlib.rcParams['font.family']='Malgun Gothic'   # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False
# ======================================================================================================================

data = pd.read_csv('AllMergeFromDonny.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
print(data.shape)
data.columns

(40000, 37)


Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')

## 데이터를 생존과 과금여부로 나눠보기

In [41]:
# 생존여부
data.loc[data['survival_time'] < 64, "survived"] = 0
data.loc[data['survival_time'] == 64, "survived"] = 1

# 과금 여부
data.loc[data['amount_spent'] == 0, "cash"] = 0
data.loc[data['amount_spent'] > 0, "cash"] = 1

# data.drop(['acc_id','survival_time','amount_spent', 'survived'], axis=1, inplace = True)

data.head()

# 나눈 것을 다시 한번 숫자 카운팅 해보기
print(data['survived'][data['survived']==0].count(),'\n',
     data['survived'][data['survived']==1].count())

print(data['cash'][data['cash']==0].count(),'\n',
     data['cash'][data['cash']==1].count())

print(data.columns)

survival_time = data[['acc_id','survival_time']]
amount_spent = data[['acc_id','amount_spent']]

data1 = data.drop(columns=['amount_spent','cash','survived','survival_time'], axis=1)
data1.shape
# ======================================================================================================================

18004 
 21996
16438 
 23562
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time', 'survived', 'cash'],
      dtype='object')


(40000, 35)

# 생존여부로 결정트리, 랜덤포레스트

In [ ]:
# ======================================================================================================================

from sklearn.model_selection import train_test_split

y = data['survived']


x = data[['acc_id','playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent',  'sell_item_cnt','amount_spent_pay', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']]


#훈련세트, 테스트세트 나누기
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

In [40]:
## DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

decision = DecisionTreeClassifier(max_depth = 5).fit(train_x,train_y)
# print(decision.score(train_x, train_y))
print(decision.score(test_x, test_y))

from sklearn.tree import export_graphviz
import graphviz
import pydot

export_graphviz(decision,
                feature_names = x.columns,
                class_names = ['Death', 'Survived'],
                out_file = 'decisionTree1.dot',
                impurity = True,
                filled = True)

# Encoding 중요
# (graph,) = pydot.graph_from_dot_file('decisionTree1.dot', encoding='utf8')

# Dot 파일을 Png 이미지로 저장
# graph.write_png('decisionTree1.png')

with open('decisionTree1.dot') as f:
    dot_graph = str(open("decisionTree1.dot", "rb").read(), 'utf8')

src = graphviz.Source(dot_graph)
src.render('payment-lable.gv', view=True)

0.6969166666666666
0.7608333333333334


## 생존/비생존으로 랜덤포레스트

In [47]:
# ======================================================================================================================
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(train_x,train_y)
# print(clf.score(train_x, train_y))
print(rfc.score(test_x, test_y))
rfc_y_score = rfc.predict_proba(test_x)

rfc_y_score = rfc.predict(data1)
rfc_y = pd.DataFrame(rfc_y_score, columns = ['survival_predict'])
print(type(rfc_y))
rfc_y.head()
# ======================================================================================================================


<class 'pandas.core.frame.DataFrame'>


,survival_predict
0,1.0
1,1.0
2,0.0
3,1.0
4,0.0


In [48]:
data_cash = pd.concat([data1, rfc_y], axis=1)
data_cash.head()
rfc_y.shape
print(len(data_cash.columns))

36


In [49]:
data_cash = data_cash.merge(survival_time, on='acc_id', how='inner')
print(data_cash.columns)
data_cash.shape
print(data_cash[data_cash['survival_predict']==1].shape, data_cash[data_cash['survival_time']==64].shape)
print(data_cash[data_cash['survival_predict']==0].shape, data_cash[data_cash['survival_time']<64].shape)

Index(['acc_id', 'playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'level',
       'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'amount_spent_pay',
       'sell_item_cnt', 'buy_item_cnt', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt_plg',
       'random_defender_cnt_plg', 'same_pledge_cnt_plg', 'temp_cnt_plg',
       'etc_cnt_plg', 'combat_play_time', 'non_combat_play_time',
       'survival_predict', 'survival_time'],
      dtype='object')
(23463, 37) (21996, 37)
(16537, 37) (18004, 37)


# 1~64로 랜덤포레스트 예측 해보기

In [57]:
# data_cash_yes = data_cash
# data_cash_yes.columns

data_cash_yes = data[['combat_play_time','death','enchant_count','etc_cnt',
          'etc_cnt_plg','exp_recovery','level','non_combat_play_time',
          'num_opponent','party_exp','play_char_cnt','pledge_combat_cnt',
          'private_shop','quest_exp','random_attacker_cnt','random_attacker_cnt_plg',
          'random_defender_cnt','random_defender_cnt_plg','same_pledge_cnt_plg',
          'sell_item_cnt','temp_cnt','temp_cnt_plg','survival_time']]

# [data_cash['survival_predict']==0]
y_data = [[i] for i in data_cash_yes['survival_time'].tolist() ]
y= y_data 

data_cash_yes.drop(['survival_time'], axis=1, inplace=True)

x_data = [ data_cash_yes.iloc[i,:].tolist() for i in range(len(data_cash_yes.index.values))]
X = x_data

from sklearn.model_selection import train_test_split

x_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) #42

# multi_variable_linear_regression.py
# 4

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\pandas\core\frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [58]:
import pandas as pd
 
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# 학습 진행
forest = RandomForestClassifier(n_estimators=100)
forest.fit(x_train, y_train)
 
# 예측 
y_pred = forest.predict(X_test)
print(y_pred)
print(list(y_test))
 
# 정확도 확인
print('정확도 :', metrics.accuracy_score(y_test, y_pred))

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


[64. 64. 64. ... 64. 64. 64.]
[[64.0], [64.0], [64.0], [64.0], [8.0], [15.0], [17.0], [64.0], [9.0], [64.0], [64.0], [15.0], [64.0], [64.0], [64.0], [8.0], [10.0], [64.0], [64.0], [42.0], [40.0], [29.0], [9.0], [64.0], [46.0], [64.0], [64.0], [64.0], [64.0], [32.0], [64.0], [22.0], [61.0], [64.0], [64.0], [64.0], [29.0], [64.0], [64.0], [64.0], [64.0], [64.0], [64.0], [64.0], [2.0], [19.0], [64.0], [42.0], [64.0], [4.0], [11.0], [64.0], [64.0], [19.0], [64.0], [32.0], [64.0], [64.0], [64.0], [64.0], [64.0], [64.0], [59.0], [41.0], [30.0], [64.0], [64.0], [64.0], [64.0], [37.0], [12.0], [64.0], [7.0], [64.0], [4.0], [10.0], [64.0], [64.0], [64.0], [1.0], [55.0], [64.0], [64.0], [64.0], [32.0], [24.0], [59.0], [64.0], [3.0], [29.0], [64.0], [64.0], [3.0], [4.0], [25.0], [64.0], [64.0], [64.0], [64.0], [64.0], [64.0], [5.0], [64.0], [37.0], [3.0], [19.0], [36.0], [64.0], [64.0], [64.0], [32.0], [17.0], [64.0], [13.0], [64.0], [19.0], [64.0], [64.0], [3.0], [64.0], [64.0], [64.0], [8.0], [

# 과금여부로 결정트리, 랜덤포레스트

In [20]:
# ======================================================================================================================


# data.survived.sum()

#훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split

y = data['cash']

# shap에서 겹치는 공통 변수
# x = data[['combat_play_time',
#  'death',
#  'enchant_count',
#  'etc_cnt',
#  'etc_cnt_plg',
#  'exp_recovery',
#  'level',
#  'non_combat_play_time',
#  'num_opponent',
#  'party_exp',
#  'play_char_cnt',
#  'pledge_combat_cnt',
#  'private_shop',
#  'quest_exp',
#  'random_attacker_cnt',
#  'random_attacker_cnt_plg',
#  'random_defender_cnt',
#  'random_defender_cnt_plg',
#  'same_pledge_cnt_plg',
#  'sell_item_cnt',
#  'temp_cnt',
#  'temp_cnt_plg']]

x = data[['acc_id','playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent',  'sell_item_cnt','amount_spent_pay', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']]







train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

## DecisionTree

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

decision = DecisionTreeClassifier(max_depth = 7).fit(train_x,train_y)
# print(decision.score(train_x, train_y))
print(decision.score(test_x, test_y))

from sklearn.tree import export_graphviz
import graphviz
import pydot

export_graphviz(decision,
                feature_names = x.columns,
                class_names = ['No_cash', 'Yes_cash'],
                out_file = 'decisionTree1.dot',
                impurity = True,
                filled = True)

# Encoding 중요
# (graph,) = pydot.graph_from_dot_file('decisionTree1.dot', encoding='utf8')

# Dot 파일을 Png 이미지로 저장
# graph.write_png('decisionTree1.png')

with open('decisionTree1.dot') as f:
    dot_graph = str(open("decisionTree1.dot", "rb").read(), 'utf8')

src = graphviz.Source(dot_graph)
src.render('payment-lable.gv', view=True)


# ======================================================================================================================
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(train_x,train_y)
# print(clf.score(train_x, train_y))
print(rfc.score(test_x, test_y))
rfc_y_score = rfc.predict_proba(test_x)

0.839
0.8576666666666667


### 랜덤포레스트 결과에서 예측값 뽑기

In [ ]:
rfc_y_score = rfc.predict(data1)
rfc_y = pd.DataFrame(rfc_y_score, columns = ['cash_predict'])
print(type(rfc_y))

In [ ]:
data_cash = pd.concat([data1, rfc_y], axis=1)
data_cash.head()
rfc_y.shape
print(len(data_cash.columns))

In [ ]:
data_cash = data_cash.merge(amount_spent, on='acc_id', how='inner')
print(data_cash.columns)
data_cash.shape
print(data_cash[data_cash['cash_predict']==1].shape, data_cash[data_cash['amount_spent']>0].shape)
print(data_cash[data_cash['cash_predict']==0].shape, data_cash[data_cash['amount_spent']==0].shape)

In [ ]:
data_cash_yes = data_cash[data_cash['cash_predict']==1]
data_cash_yes.columns

y_data = [[i] for i in data_cash_yes['amount_spent'].tolist() ]
y= y_data 

data_cash_yes.drop(['acc_id','amount_spent','cash_predict'], axis=1, inplace=True)

x_data = [ data_cash_yes.iloc[i,:].tolist() for i in range(len(data_cash_yes.index.values))]
X = x_data

from sklearn.model_selection import train_test_split

x_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) #42

# multi_variable_linear_regression.py
# 4

In [23]:
import tensorflow as tf


X = tf.placeholder(tf.float32,shape=[None,data_cash_yes.shape[1]])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([data_cash_yes.shape[1],1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X,W) + b  #

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.AdamOptimizer(learning_rate=3e-1)
train = optimizer.minimize(cost)



# RMSE( root-mean-square error ) : 평균 제곱 근 오차
# 평균 제곱근편차(RMSD) 또는 평균 제곱근오차(RMSE)는 추정값 또는 모델이 예측한 값과
# 실제 환경에서 관찰되는 값의 차이를 다룰 때 흔히 사용하는 측도이다.
# 정밀도를 표현하는데 적합하다.

targets = tf.placeholder(tf.float32,[None,1])  # 답(Y)
predictions = tf.placeholder(tf.float32,[None,1]) # 예측 값
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# start training
for step in range(10001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X:x_data, Y:y_data})
    if step % 1000 == 0:
        print(step, cost_val, W_val, b_val)

# Test step
test_predict = sess.run(hypothesis,feed_dict={X:X_test})
rmse_val = sess.run(rmse,feed_dict={targets:y_test,predictions:test_predict})
print('RMSE: {}'.format(rmse_val))


total_error = tf.reduce_sum(tf.square(tf.subtract(y_test, tf.reduce_mean(y_test))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y_test, test_predict)))
R_squared = tf.subtract(1.0,tf.divide( unexplained_error,total_error))

print('R_sqr: {}'.format(sess.run(R_squared)))

prd = sess.run(hypothesis, feed_dict = {X:X_test})

<class 'pandas.core.frame.DataFrame'>
36


Index(['acc_id', 'playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'level',
       'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'amount_spent_pay',
       'sell_item_cnt', 'buy_item_cnt', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt_plg',
       'random_defender_cnt_plg', 'same_pledge_cnt_plg', 'temp_cnt_plg',
       'etc_cnt_plg', 'combat_play_time', 'non_combat_play_time',
       'cash_predict'],
      dtype='object')

In [ ]:
import pandas as pd
 
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
 
if __name__ == '__main__':
    iris = datasets.load_iris()
    print('아이리스 종류 :', iris.target_names)
    print('target : [0:setosa, 1:versicolor, 2:virginica]')
    print('데어터 수 :', len(iris.data))
    print('데이터 열 이름 :', iris.feature_names)
 
    # iris data Dataframe으로
    data = pd.DataFrame(
        {
            'sepal length': iris.data[:, 0],
            'sepal width': iris.data[:, 1],
            'petal length': iris.data[:, 2],
            'petal width': iris.data[:, 3],
            'species': iris.target
        }
    )
    print(data.head())
 
    x = data[['sepal length', 'sepal width', 'petal length', 'petal width']]
    y = data['species']
 
    # 테스트 데이터 30%
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    print(len(x_train))
    print(len(x_test))
    print(len(y_train))
    print(len(y_test))
 
    # 학습 진행
    forest = RandomForestClassifier(n_estimators=100)
    forest.fit(x_train, y_train)
 
    # 예측
    y_pred = forest.predict(x_test)
    print(y_pred)
    print(list(y_test))
 
    # 정확도 확인
    print('정확도 :', metrics.accuracy_score(y_test, y_pred))